In [ ]:
!pip install faiss-cpu

# Install Ollama Python SDK
!pip install ollama

# Install Numpy
!pip install numpy

# (Optional) Install Transformers if you need it for other models
!pip install sentence_transformers

# (Optional) Install Pandas if you need it for data handling
!pip install pandas

In [3]:
import os

# Directory containing text files
directory = "../data/"

# Read all text files and combine them
all_texts = []
for filename in os.listdir(directory):
    if filename.endswith(".txt"):
        print(filename)
        with open(os.path.join(directory, filename), 'r', encoding='utf-8') as file:
            all_texts.append(file.read())

CMU_Traditions.txt
CMU_Wikipedia.txt
Transportation_Pitts_Wikipedia.txt
Pittsburgh_Wikipedia.txt
festivals.txt
Visit_Pittsburgh.txt
current_affairs.txt
City_Of_Pittsbugh.txt
LocalService_Regulations.txt
opera_events.txt
ISP_Regulations.txt
PIRATES.txt
pgh_cultural_trust.txt
PENGUINS.txt
CMU_History.txt
UF_Regulations.txt
carnegie_museums.txt
Bridges_Wikipedia.txt
STEELERS.txt
Events_Pittsburgh.txt
sports_teams.txt
wikis.txt
Events_CMU.txt
annual_fests.txt
CMU_Events_Extra.txt
sports_schedules.txt
History_WIkipedia.txt
museums_list.txt
CMU.txt
Visit_Pitts 1.txt
symphony.txt
Culture_Pitts_wikipedia.txt
Parking_Regulations.txt
Amusement_Regulations.txt
CMU_Welcome_To_Pittsburgh.txt
Payroll_Regulations.txt


In [5]:
def chunk_text(text, max_length=128):
    words = text.split()
    chunks = []

    for i in range(0, len(words), max_length):
        chunks.append(" ".join(words[i:i + max_length]))

    return chunks

# Create chunks from each file content
text_chunks = []
for text in all_texts:
    text_chunks.extend(chunk_text(text, max_length=256))
print(len(text_chunks))

2909


In [7]:
import faiss
from sentence_transformers import SentenceTransformer

# Load a pre-trained model for embeddings
model = SentenceTransformer('all-MiniLM-L6-v2')

/opt/anaconda3/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/opt/anaconda3/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [9]:
doc_embeddings = model.encode(text_chunks, convert_to_tensor=True)
embedding_dim = doc_embeddings.shape[1]

index = faiss.IndexFlatL2(embedding_dim)
index.add(doc_embeddings.cpu().numpy())

faiss.write_index(index, 'faiss_index.index')

In [11]:
def retrieve_documents(query, k=3):
    query_embedding = model.encode([query])
    distances, indices = index.search(query_embedding, k)
    matching_chunks = [text_chunks[i] for i in indices[0]]
    return matching_chunks

In [433]:
def generate_qa_pairs(context, num_questions=1):
    # Define the prompt to instruct the model to generate questions based on the context
    prompt = f"""
    Read the following context and generate {num_questions} possible question-answer pairs:

    Context:
    {context}

    Provide the output in the following format:
    Q1: <Question>
    A1: <Answer1>
    Q2: <Question>
    A2: <Answer2>
    ...
    
    Here are examples of the types of questions and answers you should generate:
    
    1. Give direct and concise answers, preferably in least words.
    For example:
    Q1: What are the operating hours of the CMU Pantry?
    A1: 2 p.m. to 5 p.m.
    
    3. Ensure that each question can be answered from the context and is not vague or ambiguous.
    4. Generate multiple unrelated questions for similar topics through paraphrasing and identifying key information.
    6. Reduce questions for which no context is available.
    7. Each question should be answerable independently, do not return questions dependent on any previous answer.
    For example:
    Q3: When is the X event happening?
    should not be followed by:
    Q4. What is it's location?
    Instead, it should be followed by:
    Q4: What is the location of the X event?

    8. While answering a question about some event X, do not use "it" or "the event".
    For example:
    Q3: How long does the Gender in Process event last?
    Wrong answer: This event runs from 3:30 to 5 p.m.
    Correct answer: 3:30 to 5 p.m.

    Phrase the answer well - don't give answers in points like "Week 12, · Sun 11/21, · 8:15 PM EST.", instead the answer should be "Sunday, November 21st, 2024, 8:15 PM EST"
    """

    # Send the prompt to the Ollama server
    url = "http://localhost:11434/api/generate"
    payload = {
        "model": "llama3.2",
        "prompt": prompt,
        "stream": False,  # Set to True if you want the response streamed back
        # "top_p": 0.9
    }
    
    try:
        response = requests.post(url, json=payload)
        response.raise_for_status()  # Raise an error for bad responses
        result = response.json().get('response', "")
        
        # Parse the output to separate questions and answers
        questions, answers = [], []
        lines = result.strip().split("\n")
        
        for line in lines:
            if line.startswith("Q"):
                questions.append(line.split(": ", 1)[1].strip())
            elif line.startswith("A"):
                answers.append(line.split(": ", 1)[1].strip())
        
        return questions, answers

    except requests.ConnectionError:
        print("Connection error: Could not connect to the Ollama server.")
        return [], []
    except requests.HTTPError as http_err:
        print(f"HTTP error occurred: {http_err}")
        return [], []
    except Exception as err:
        print(f"An error occurred: {err}")
        return [], []

In [ ]:
import pandas as pd
qa_df = pd.DataFrame(columns=["question", "answer", "context"])

questions_list = []
answers_list = []
count_chunks = 641

# for chunk in text_chunks:
#     # print(chunk)
#     fq = open("questions" + str(count_chunks) + ".txt", "w+")
#     fa = open("answers" + str(count_chunks) + ".txt", "w+")
#     questions, answers = generate_qa_pairs(chunk)
#     for question in questions:
#         fq.write(question + "\n")
#     for answer in answers:
#         fa.write(answer + "\n")
#     # questions_list.extend(questions)
#     # answers_list.extend(answers)
#     count_chunks+=1

for chunk in text_chunks:
    questions, answers = generate_qa_pairs(chunk)
    print(questions,answers)
    # for question in questions:
    #     retrieved_chunks = retrieve_documents(question)
    #     context = " ".join(retrieved_chunks)
    #     answer = ollama_generate(question, context)
    #     new_row = pd.DataFrame({"question": [question], "answer": [answer], "context": [context]})
    #     qa_df = pd.concat([qa_df, new_row], ignore_index=True)

# for i in range(len(questions_list)):
#     print(f"Question: {questions_list[i]}\nAnswer: {answers_list[i]}\n")
# print(questions, answers)

In [ ]:
qa_df

In [123]:
qa_df.to_csv("qa_pairs_sports_schedule.csv", index=False)